### Курсовой проект

Задача: построить рекомендательную систему по товарам.

Целевая метрика precision@5

Hints:

Сначала просто попробуйте разные параметры MainRecommender:

N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)
Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)
Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)
Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)

#### Реализация (пайплайн)
загружаем данные
разбиваем на трейн/тесты в соответствии с 2 уровнями
осуществляем предфильрацию
обучаем рекоммендер первого уровня. при обучении используем tfidf-взвешивание, берем own_rec - прочие были отметены опытным путем
#### готовим фичи для товаров:
эмбеддинги
цена
среднее кол-во товара в корзине
накопительная выручка по товару
кол-во товаров в той же категории
кол-во дней с последней продажи. если продаж за период не было, то берем кол-во дней в периоде и умножаем на 2 (типа вес)
оставшиеся фичи преобразуем в категориальные
#### готовим фичи для юзеров:
эмбеддинги
средний чек
дней с последней покупки. если покупок за период не было, то берем кол-во дней в периоде и умножаем на 2 (типа вес)
преобразуем возраст, средний доход, размер дома и кол-во детей в числовой формат
оставшиеся фичи преобразуем в категориальные
обучаем модель второго уровня. в качестве результата берем скор предикта.
по скорам отбираем для юзера рекомендованные товары (100)
по бизнес-требованиям из них отбираем по 5 товаров
считаем метрику
с помощью обученной модели считаем предикт для тестовых данных, считаем метрику.

In [ ]:
import pandas as pd
import numpy as np

from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.metrics import precision_at_k, recall_at_k, money_precision_at_k
from src.utils import prefilter_items, get_users_features, get_items_features, get_recommendation_5
from src.recommenders import MainRecommender

import warnings
warnings.filterwarnings("ignore")

from tqdm import tqdm
tqdm.pandas()

In [195]:
path = 'C:\\Users\Матвей\\'
path_data = path + 'retail_train.csv'  # ниже загружаю уже с расчитанной ценой
path_features = path + 'product.csv'
path_user = path + 'hh_demographic.csv'

data = pd.read_csv(path_data)
item_features = pd.read_csv(path_features)
user_features = pd.read_csv(path_user)

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

In [196]:
test_path = path + 'retail_test1.csv'
test_data = pd.read_csv(test_path)

In [197]:
data.shape, test_data.shape

((2396804, 12), (88734, 12))

In [198]:
#сразу считаем цены - они нам потребуются далее
prices = data.groupby(['item_id'])['sales_value'].mean().reset_index()
sales_qty = data.groupby(['item_id'])['quantity'].mean().reset_index()
prices = prices.merge(sales_qty, on='item_id', how='left')
prices['price'] = [prices.iloc[i]['sales_value'] / prices.iloc[i]['quantity']\
                   if prices.iloc[i]['quantity'] > 0 else 0 for i in prices['item_id'].index]
prices.drop(columns=['sales_value', 'quantity'], axis=1, inplace=True)

In [199]:
data = data.merge(prices, on='item_id', how='left')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,2.385178
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,0.945892


In [200]:
# Схема разбиения: все данные -> 6 недель -> 3 недели
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] <= data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  
data_val_lvl_2 = data[data['week_no'] > data['week_no'].max() - val_lvl_2_size_weeks]

In [201]:
# Тренировочные данные для 1 уровня
data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,2.385178
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,0.945892


In [202]:
# Предфильтрация
n_items_before = data_train_lvl_1['item_id'].nunique()
data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)
n_items_after = data_train_lvl_1['item_id'].nunique()
print(f'Decreased # items from {n_items_before} to {n_items_after}')

Decreased # items from 83685 to 5001


In [203]:
items_sold = data_train_lvl_1['item_id'].unique().tolist()
print(f'Отобрано категорий - {item_features[item_features["item_id"].isin(items_sold)]["sub_commodity_desc"].nunique()}')

Отобрано категорий - 815


In [209]:
recommender = MainRecommender(data_train_lvl_1, weighting=True)

In [211]:
# actual - фактически купленные товары
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [206]:
N_predict = 200 # Кол-во рекомендаций для юзера
users = set(data_train_lvl_1['user_id'].unique().tolist())
top_popular_items = recommender.overall_top_purchases[:N]

In [ ]:
# Строим рекомендации
result_lvl_1['own_recommendations'] = result_lvl_1['user_id'].progress_apply(lambda x: \
                                                                             recommender.get_own_recommendations(x, N=N))

In [212]:
result_lvl_1.tail(2)

,user_id,actual
2195,2499,"[861282, 921744, 1050968, 13842089, 828837, 86..."
2196,2500,"[856455, 902192, 903476, 931672, 936634, 95170..."


In [213]:
# Отберем уникальных юзеров для обучения 2 уровня
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']
# Пока только warm start - для новых используем бейзлайн
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

In [214]:
# добавляем рекомендации с первого уровня
users_lvl_2 = users_lvl_2.merge(result_lvl_1, on='user_id', how='left')
users_lvl_2.tail(2)

,user_id,actual
2192,903,"[923746, 1005274, 1070820, 6961519]"
2193,1276,"[834484, 855672, 860776, 879528, 954355, 95802..."


In [ ]:
user_features_upd = get_users_features(user_features, data_train_lvl_2)
user_features_upd.head(2)

In [ ]:
item_features_upd = get_items_features(item_features, data_train_lvl_2)
item_features_upd.head(2)

In [ ]:
#Добавим цены - надо бы внести в функцию подготовки фичей по товарам!
item_features_upd = item_features_upd.merge(prices, on='item_id', how='left')

In [ ]:
s = users_lvl_2.apply(lambda x: pd.Series(x['own_recommendations']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'
users_lvl_2 = users_lvl_2.drop('own_recommendations', axis=1).join(s)
users_lvl_2 = users_lvl_2.drop('actual', axis=1)
users_lvl_2['drop'] = 1  # фиктивная переменная

In [ ]:
targets_lvl_2 = data_train_lvl_2[['user_id','item_id']].copy()
targets_lvl_2['target'] = 1  # тут покупки 
targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')
targets_lvl_2['target'].fillna(0, inplace= True) # тут не было покупок
targets_lvl_2.drop('drop', axis=1, inplace=True)

In [ ]:
targets_lvl_2 = targets_lvl_2.merge(user_features_upd, on='user_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(item_features_upd, on='item_id', how='left')

In [ ]:
cat_feats = ['marital_status_code', 'homeowner_desc', 'hh_comp_desc', 'manufacturer', 'department', 'brand', 
             'commodity_desc', 'sub_commodity_desc', 'curr_size_of_product']

In [ ]:
%%time

lgb = LGBMClassifier(objective='binary', max_depth=7, categorical_column=cat_feats)
lgb.fit(X_train, y_train)

In [ ]:
targets_lvl_3 = data_val_lvl_2[['user_id', 'item_id']].copy()
targets_lvl_3.drop_duplicates(keep='first', inplace=True)

In [ ]:
targets_lvl_3 = targets_lvl_3.merge(user_features_upd, on='user_id', how='left')
targets_lvl_3 = targets_lvl_3.merge(item_features_upd, on='item_id', how='left')
targets_lvl_3.shape

In [ ]:
preds = lgb.predict(targets_lvl_3)
test_preds_proba = lgb.predict_proba(targets_lvl_3)

In [ ]:
targets_lvl_3['res'] = test_preds_proba[:, 1]
targets_lvl_3.head(2)

In [ ]:
targets_lvl_3.sort_values(['user_id', 'res'], ascending=False, inplace=True)
recs = targets_lvl_3.groupby('user_id')['item_id']

recomendations = []
for user, preds in recs:
    recomendations.append({'user_id': user, 'recomendations': preds.tolist()})

In [ ]:
recomendations = pd.DataFrame(recomendations)

result_lvl_2 = data_val_lvl_2.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_2.columns=['user_id', 'actual']

result_lvl_2 = result_lvl_2.merge(recomendations, on='user_id', how='left')
result_lvl_2.head(2)

In [ ]:
top_valued_items = prices[(prices['price'] > 7)]
top_valued_items = top_valued_items.sort_values(by='price', ascending=False, na_position='last')
top_valued_items_list = top_valued_items['item_id'].tolist()

In [ ]:
# отфильтруем < $1
top_popular_items_m = prices[(prices['price'] > 1) & (prices['item_id'].isin(top_popular_items))]['item_id'].tolist() 
print(f'Популярных товаров до фильтрации - {len(top_popular_items)}, после - {len(top_popular_items_m)}')

In [ ]:
top_valued_items_list = top_valued_items['item_id'].tolist()

In [ ]:
result_lvl_2['rec'] = result_lvl_2['user_id'].progress_apply\
        (lambda x: get_recommendation_5(x, result_lvl_2, item_features, top_popular_items_m, top_valued_items_list)[0])

In [ ]:
result_lvl_2.progress_apply(lambda row: money_precision_at_k(row['rec'], row['actual'], prices), axis=1).mean()

Проверяем на тесте

In [ ]:
test_data_upd = test_data[['user_id', 'item_id']].copy()
test_data_upd.drop_duplicates(keep='first', inplace=True)

In [ ]:
test_data_upd = test_data_upd.merge(user_features_upd, on='user_id', how='left')
test_data_upd = test_data_upd.merge(item_features_upd, on='item_id', how='left')
test_data_upd.shape

In [ ]:
test_preds_proba_2 = lgb.predict_proba(test_data_upd)[:, 1]

In [ ]:
test_data_upd['proba'] = test_preds_proba_2
test_data_upd = test_data_upd[test_data_upd['price'] > 1]

In [ ]:
result = test_data_upd.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

In [ ]:
test_data_upd.sort_values(['user_id', 'proba'], ascending=False, inplace=True)
recs = test_data_upd.groupby('user_id')['item_id']

recomendations = []
for user, preds in recs:
    recomendations.append({'user_id': user, 'recomendations': preds.tolist()})

In [ ]:
recomendations = pd.DataFrame(recomendations)

In [ ]:
result = result.merge(recomendations, on='user_id', how='left')
result.head(2)

In [ ]:
result['rec'] = result['user_id'].progress_apply\
        (lambda x: get_recommendation_5(x, result, item_features, top_popular_items_m, top_valued_items_list)[0])

In [ ]:
result['pres'] = result.progress_apply(lambda row: \
                                                money_precision_at_k(row['rec'], row['actual'], prices), axis=1)#.mean()

In [ ]:
result['pres'].mean()